In [1]:
%load_ext autoreload
%autoreload 2
import edc.utils.llm_utils as llm_utils
from peft import PeftModel, PeftConfig
from edc.utils.e5_mistral_utils import MistralForSequenceEmbedding
from edc.extract import Extractor
from edc.schema_definition import SchemaDefiner
from edc.schema_canonicalization import SchemaCanonicalizer
from edc.schema_retriever import SchemaRetriever
from edc.edc_framework import EDC
from edc.utils.e5_mistral_utils import MistralForSequenceEmbedding
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
!pwd

import sys

sys.path.append("../")

/home/users/bowen/coding/edc


In [4]:
extraction_llm_name = "mistral7b"
schema_definition_llm_name = "mistral7b"
schema_canonicalization_verifier_model_name = "mistral7b"

edc_configuration = {'extractor_llm': extraction_llm_name, 'schema_definition_llm': schema_definition_llm_name, 'canonicalization_verifier_model': schema_canonicalization_verifier_model_name}

In [5]:
edc = EDC(edc_configuration)

Models used: {'mistral7b'}
Loading models...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [6]:
edc.extract_kg(['Bowen is a student at National University of Singapore'])

FileNotFoundError: [Errno 2] No such file or directory: './prompt_templates/few_shot_examples.txt'

In [2]:
mistral7b_model, mistral7b_tokenizer = llm_utils.load_model('mistral7b')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
embedding_model = MistralForSequenceEmbedding.from_pretrained("intfloat/e5-mistral-7b-instruct", device_map="auto")
embedding_tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-mistral-7b-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
extractor = Extractor(mistral7b_model, mistral7b_tokenizer)

In [4]:
input_text = "Bowen is a student at National University of Singapore."
oie_few_shot_examples_str = open('./edc/prompt_templates/few_shot_examples.txt').read()
oie_few_shot_prompt_template_str = open('./edc/prompt_templates/extract_template_few_shot.txt').read()

In [5]:
oie_triplets = extractor.extract(input_text, oie_few_shot_examples_str, oie_few_shot_prompt_template_str)

[{'role': 'user', 'content': 'Your task is to transform the given text into a semantic graph in the form of a list of triples. The triples must be in the form of [Entity1, Relationship, Entity2]. In your answer, please strictly only include the triples and do not include any explanation or apologies.\n\nHere are some examples:\nExample 1:\nText: Spodnje Hoče () is a settlement in and the administrative centre of the Municipality of Hoče–Slivnica in northeastern Slovenia.\nTriplets: [[\'Spodnje Hoče\', \'located in the administrative territorial entity\', \'Municipality of Hoče–Slivnica\'], [\'Municipality of Hoče–Slivnica\', \'contains settlement\', \'Spodnje Hoče\'], [\'Municipality of Hoče–Slivnica\', \'country\', \'Slovenia\'], [\'Slovenia\', \'contains administrative territorial entity\', \'Municipality of Hoče–Slivnica\']]\n\nExample 2:\nText: Herbert Blankenhorn (15 December 1904 in Mülhausen\xa0– 10 August 1991 in Badenweiler) was a German diplomat.\nTriplets: [[\'Herbert Blanke

In [6]:
oie_triplets

[['Bowen', 'student of', 'National University of Singapore']]

In [7]:
schema_definition_few_shot_prompt_template_str = open('./edc/prompt_templates/schema_definition_template.txt').read()

In [8]:
schema_definer = SchemaDefiner(mistral7b_model, mistral7b_tokenizer)

In [16]:
open_schema_definition_dict = schema_definer.define_schema(input_text, oie_triplets, "", schema_definition_few_shot_prompt_template_str)

[{'role': 'user', 'content': "You will be given a piece of text and a list of relational triples in the format of [Subject, Relation, Object] extracted from the text. For each relation present in the triples, your task is to write a description to express the meaning of the relation.\n\nHere are some examples:\n\nExample 1:\nText: Romany Love is a 1931 British musical film directed by Fred Paul and starring Esmond Knight, Florence McHugh and Roy Travers.\nTriples: [['Romany Love', 'publication date', '1931'], ['Romany Love', 'genre', 'musical film'], ['Romany Love', 'director', 'Fred Paul'], ['Romany Love', 'cast member', 'Esmond Knight']]\nRelations: ['publication date', 'genre', 'director', 'cast member']\nAnswer: \npublication date: The subject entity was published or released on the date specified by the object entity.\ngenre: The subject entity belongs to the genre specified by the object entity.\ndirector: The subject entity was directed by the person or entity specified by the o

In [18]:
target_schema = {'student': 'The subject entity studies at the institution or educational body specified by the object entity.', 'gender': 'The subject entity has the gender specified by the object entity.'}

In [33]:
canonicalize_verify_prompt_template_str = open('./edc/prompt_templates/canonicalize_verify_template.txt').read()

In [36]:
canonicalizer = SchemaCanonicalizer(target_schema, embedding_model, embedding_tokenizer, mistral7b_model, mistral7b_tokenizer)

In [37]:
original = {'student of': ['The subject entity is a student of the institution or educational body specified by the object entity.']}
canonicalizer.canonicalize(input_text, oie_triplets[0], open_schema_definition_dict, canonicalize_verify_prompt_template_str)

["<s>[INST] Given the following text and a relational triplet extracted from it:\n\nText: Bowen is a student at National University of Singapore.\nTriplet: ['Bowen', 'student of', 'National University of Singapore']\n\nThe relation 'student of' in the triplet is defined as '['The subject entity is a student of the institution or educational body specified by the object entity.</s>']' In this context, is there any relation appropriate to replace it? Please answer by providing only the letter of your choice.\n\nChoices:\nA. 'student': The subject entity studies at the institution or educational body specified by the object entity.\nB. 'gender': The subject entity has the gender specified by the object entity.\nC. None of the above.\n [/INST]Answer: "]
A. 'student': The subject entity is a student at the institution or educational body specified by the object entity.

Explanation: The given triplet correctly represents the relationship between Bowen and National University of Singapore, w